In [2]:
import pickle
import os 
import pandas as pd 

pickle_file = open("../data/car_number_1.pkl", "rb")
unpickled = pickle.load(pickle_file)

basic_info = unpickled[0]
basic_info.update(unpickled[1])
additional_info = unpickled[2]

print("=== BASIC INFO ===\n", basic_info.keys())
print("\n=== ADDITIONAL INFO ===\n", additional_info)

=== BASIC INFO ===
 dict_keys(['url', 'image', 'brand', 'model', 'manufacturer', 'productionDate', 'fuelType', 'bodyType', 'vehicleEngine', 'mileageFromOdometer', 'name', 'Plivajući zamajac', 'Emisiona klasa motora', 'Pogon', 'Menjač', 'Broj vrata', 'Broj sedišta', 'Strana volana', 'Klima', 'Boja', 'Materijal enterijera', 'Boja enterijera', 'Registrovan do', 'Poreklo vozila', 'Oštećenje'])

=== ADDITIONAL INFO ===
 ['Airbag za vozača', 'Airbag za suvozača', 'Bočni airbag', 'ABS', 'ESP', 'Kodiran ključ', 'Blokada motora', 'Centralno zaključavanje', 'Metalik boja', 'Branici u boji auta', 'Servo volan', 'Multifunkcionalni volan', 'Tempomat', 'Daljinsko zaključavanje', 'Putni računar', 'Tonirana stakla', 'Električni podizači', 'Električni retrovizori', 'Sedišta podesiva po visini', 'Svetla za maglu', 'Senzori za svetla', 'Senzori za kišu', 'Parking senzori', 'Aluminijumske felne', 'Navigacija', 'Radio CD', 'Naslon za ruku', 'Hands free', 'Kožni volan', 'MP3', 'Utičnica od 12V', 'Otvor za s

In [3]:
def generate_price(basic_info, scale=1e6):
    
    mileage_str_raw = basic_info['mileageFromOdometer'] 
    mileage_str = mileage_str_raw[ : -4]
    mileage = scale * 1 / float(mileage_str)

    return mileage



def collect_raw_data(data_folder, max_samples=1000):

    raw_data = []
    file_names = os.listdir(data_folder)

    for i, file_name in enumerate(file_names):
        
        if i >= max_samples:
            break 

        file_path = os.path.join(data_folder, file_name)
        pickle_file = open(file_path, "rb")
        unpickled = pickle.load(pickle_file)

        basic_info = unpickled[0]
        basic_info.update(unpickled[1])
        additional_info = unpickled[2]

        price = generate_price(basic_info)

        raw_data.append({
            'features': basic_info,
            'price': price 
            })

    return raw_data

In [5]:
data_folder = "../data"

raw_data = collect_raw_data(data_folder)
# raw_data

In [6]:
keys_to_keep = [
    'brand', 'productionDate', 'bodyType', 'fuelType', 
    'vehicleEngine', 'Broj sedišta', 'Oštećenje',
    'Menjač', 'Klima'
]
raw_data[0]['features'].keys()

dict_keys(['url', 'image', 'brand', 'model', 'manufacturer', 'productionDate', 'fuelType', 'bodyType', 'vehicleEngine', 'mileageFromOdometer', 'name', 'Plivajući zamajac', 'Emisiona klasa motora', 'Pogon', 'Menjač', 'Broj vrata', 'Broj sedišta', 'Strana volana', 'Klima', 'Boja', 'Materijal enterijera', 'Boja enterijera', 'Registrovan do', 'Poreklo vozila', 'Oštećenje'])

In [11]:
# key = 'model'
values_to_keep = []

for i in range(4):
    
    curr_features = [
        raw_data[i]['features'][key] for key in keys_to_keep
    ] 

    curr_features.append(raw_data[i]['price'])
    
    values_to_keep.append(curr_features)

df = pd.DataFrame(values_to_keep, columns=keys_to_keep+['price'])
df

,brand,productionDate,bodyType,fuelType,vehicleEngine,Broj sedišta,Oštećenje,Menjač,Klima,price
0,Peugeot,2009,Limuzina,Dizel,80.00 KWT,5 sedišta,Nije oštećen,Manuelni 5 brzina,Automatska klima,5297.648374
1,Renault,2020,Džip/SUV,Benzin,103.00 KWT,5 sedišta,Nije oštećen,Manuelni 6 brzina,Automatska klima,200000.000000
2,Mercedes Benz,2015,Limuzina,Dizel,80.00 KWT,4 sedišta,Nije oštećen,Automatski,Automatska klima,6756.756757
3,Fiat,2006,Hečbek,Dizel,51.00 KWT,5 sedišta,Nije oštećen,Manuelni 5 brzina,Manuelna klima,5586.592179
